<a href="https://colab.research.google.com/github/Nageswar-Sahoo/Computer-Vision-Project/blob/main/S10/Assignment10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone https://github.com/Nageswar-Sahoo/Computer-Vision-Project.git

fatal: destination path 'Computer-Vision-Project' already exists and is not an empty directory.


In [11]:
!pip install -U git+https://github.com/albumentations-team/albumentations
import albumentations 
from albumentations.pytorch import ToTensorV2

  Cloning https://github.com/albumentations-team/albumentations to /tmp/pip-req-build-jzt4yg5c
  Running command git clone -q https://github.com/albumentations-team/albumentations /tmp/pip-req-build-jzt4yg5c


In [12]:
ls

Computer-Vision-Project/  sample_data/


In [2]:
cd Computer-Vision-Project/template

/content/Computer-Vision-Project/template


In [13]:
cd ..

/


In [14]:
cd /content

/content


In [15]:
rm -rf /Computer-Vision-Project/template

In [16]:
ls

Computer-Vision-Project/  sample_data/


In [17]:
!wget -nc http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip tiny-imagenet-200.zip

Streaming output truncated to the last 5000 lines.
  inflating: tiny-imagenet-200/val/images/val_3979.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3963.JPEG  
  inflating: tiny-imagenet-200/val/images/val_7199.JPEG  
  inflating: tiny-imagenet-200/val/images/val_2752.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9687.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9407.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3603.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3412.JPEG  
  inflating: tiny-imagenet-200/val/images/val_6982.JPEG  
  inflating: tiny-imagenet-200/val/images/val_8496.JPEG  
  inflating: tiny-imagenet-200/val/images/val_7332.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9241.JPEG  
  inflating: tiny-imagenet-200/val/images/val_4196.JPEG  
  inflating: tiny-imagenet-200/val/images/val_5980.JPEG  
  inflating: tiny-imagenet-200/val/images/val_6697.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9969.JPEG  
  inflating: tiny-ima

In [47]:
ls tiny-imagenet-200/test/images/

test_0.JPEG     test_2800.JPEG  test_4600.JPEG  test_6400.JPEG  test_8200.JPEG
test_1000.JPEG  test_2801.JPEG  test_4601.JPEG  test_6401.JPEG  test_8201.JPEG
test_1001.JPEG  test_2802.JPEG  test_4602.JPEG  test_6402.JPEG  test_8202.JPEG
test_1002.JPEG  test_2803.JPEG  test_4603.JPEG  test_6403.JPEG  test_8203.JPEG
test_1003.JPEG  test_2804.JPEG  test_4604.JPEG  test_6404.JPEG  test_8204.JPEG
test_1004.JPEG  test_2805.JPEG  test_4605.JPEG  test_6405.JPEG  test_8205.JPEG
test_1005.JPEG  test_2806.JPEG  test_4606.JPEG  test_6406.JPEG  test_8206.JPEG
test_1006.JPEG  test_2807.JPEG  test_4607.JPEG  test_6407.JPEG  test_8207.JPEG
test_1007.JPEG  test_2808.JPEG  test_4608.JPEG  test_6408.JPEG  test_8208.JPEG
test_1008.JPEG  test_2809.JPEG  test_4609.JPEG  test_6409.JPEG  test_8209.JPEG
test_1009.JPEG  test_280.JPEG   test_460.JPEG   test_640.JPEG   test_820.JPEG
test_100.JPEG   test_2810.JPEG  test_4610.JPEG  test_6410.JPEG  test_8210.JPEG
test_1010.JPEG  test_2811.JPEG  test_4611.JPEG  test_

In [44]:
cat tiny-imagenet-200/train/n01443537/n01443537_boxes.txt

n01443537_0.JPEG	0	10	63	58
n01443537_1.JPEG	14	2	54	49
n01443537_2.JPEG	0	0	63	63
n01443537_3.JPEG	0	16	48	54
n01443537_4.JPEG	15	0	51	63
n01443537_5.JPEG	0	0	63	63
n01443537_6.JPEG	13	21	52	41
n01443537_7.JPEG	0	0	63	63
n01443537_8.JPEG	0	14	37	36
n01443537_9.JPEG	16	18	56	53
n01443537_10.JPEG	1	17	58	56
n01443537_11.JPEG	0	0	61	52
n01443537_12.JPEG	4	3	61	63
n01443537_13.JPEG	51	7	63	16
n01443537_14.JPEG	34	21	63	54
n01443537_15.JPEG	0	1	52	56
n01443537_16.JPEG	0	14	32	36
n01443537_17.JPEG	0	8	63	58
n01443537_18.JPEG	0	21	41	53
n01443537_19.JPEG	12	6	63	59
n01443537_20.JPEG	17	3	52	43
n01443537_21.JPEG	26	25	50	48
n01443537_22.JPEG	13	39	37	50
n01443537_23.JPEG	0	12	63	63
n01443537_24.JPEG	0	17	63	59
n01443537_25.JPEG	0	2	63	63
n01443537_26.JPEG	0	5	63	63
n01443537_27.JPEG	21	1	63	29
n01443537_28.JPEG	0	0	63	40
n01443537_29.JPEG	0	0	63	63
n01443537_30.JPEG	0	13	63	50
n01443537_31.JPEG	2	0	63	63
n01443537_32.JPEG	0	9	63	43
n01443537_33.JPEG	0	6	48	51
n01443537_34.JPEG	0	15	63	54
n014

In [27]:
ls -lh tiny-imagenet-200/test/images  | wc -l

10001


In [29]:
ls -lh tiny-imagenet-200/val/images  | wc -l

10001


In [36]:
ls tiny-imagenet-200/train/*/*  | wc -l

100600


In [38]:
100600/30*100

335333.3333333334

In [35]:
ls tiny-imagenet-200/train/*/*

Streaming output truncated to the last 5000 lines.
n04398044_170.JPEG  n04398044_283.JPEG  n04398044_396.JPEG  n04398044_58.JPEG
n04398044_171.JPEG  n04398044_284.JPEG  n04398044_397.JPEG  n04398044_59.JPEG
n04398044_172.JPEG  n04398044_285.JPEG  n04398044_398.JPEG  n04398044_5.JPEG
n04398044_173.JPEG  n04398044_286.JPEG  n04398044_399.JPEG  n04398044_60.JPEG
n04398044_174.JPEG  n04398044_287.JPEG  n04398044_39.JPEG   n04398044_61.JPEG
n04398044_175.JPEG  n04398044_288.JPEG  n04398044_3.JPEG    n04398044_62.JPEG
n04398044_176.JPEG  n04398044_289.JPEG  n04398044_400.JPEG  n04398044_63.JPEG
n04398044_177.JPEG  n04398044_28.JPEG   n04398044_401.JPEG  n04398044_64.JPEG
n04398044_178.JPEG  n04398044_290.JPEG  n04398044_402.JPEG  n04398044_65.JPEG
n04398044_179.JPEG  n04398044_291.JPEG  n04398044_403.JPEG  n04398044_66.JPEG
n04398044_17.JPEG   n04398044_292.JPEG  n04398044_404.JPEG  n04398044_67.JPEG
n04398044_180.JPEG  n04398044_293.JPEG  n04398044_405.JPEG  n04398044_68.JPEG
n04398044_181.

In [4]:
!wget -nc http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip tiny-imagenet-200.zip
!rm -r ./tiny-imagenet-200/test

Streaming output truncated to the last 5000 lines.
  inflating: tiny-imagenet-200/val/images/val_3979.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3963.JPEG  
  inflating: tiny-imagenet-200/val/images/val_7199.JPEG  
  inflating: tiny-imagenet-200/val/images/val_2752.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9687.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9407.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3603.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3412.JPEG  
  inflating: tiny-imagenet-200/val/images/val_6982.JPEG  
  inflating: tiny-imagenet-200/val/images/val_8496.JPEG  
  inflating: tiny-imagenet-200/val/images/val_7332.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9241.JPEG  
  inflating: tiny-imagenet-200/val/images/val_4196.JPEG  
  inflating: tiny-imagenet-200/val/images/val_5980.JPEG  
  inflating: tiny-imagenet-200/val/images/val_6697.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9969.JPEG  
  inflating: tiny-ima

In [5]:

!python3 utils/val_format.py
!find . -name "*.txt" -delete
!mkdir models
!cp -r tiny-imagenet-200 tiny-224
!python3 utils/resize.py

In [11]:
data_dir = 'tiny-imagenet-200/'
num_workers = {'train' : 100,'val'   : 0,'test'  : 0}
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
    ])
}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) 
                  for x in ['train', 'val','test']}
dataloaders = {x: data.DataLoader(image_datasets[x], batch_size=100, shuffle=True, num_workers=num_workers[x])
                  for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 100 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [19]:
import torch, os
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.models as models
from parse_config import ConfigParser
from trainer import Trainer
from utils import prepare_device
import model.loss as module_loss
import model.metric as module_metric
import torch.optim as optim
import logging
import numpy as np
import torch
import model.customresnet as module_resnet
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
import torch
import data_loader.data_loaders as data_loaders
%matplotlib inline

In [13]:
dataloaders['test']

In [16]:
dataloaders['val']

In [17]:
dataloaders['train']

In [21]:
    logger = logging.getLogger("trian")
    # Read the config.json
    config = ConfigParser.from_args()

    data_loader = dataloaders['train']

    valid_data_loader = dataloaders['val']

    # build model architecture, then print to console
    model = models.resnet18()
    model_ft = models.resnet18()
    #Finetune Final few layers to adjust for tiny imagenet input
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    model.fc.out_features = 200

    logger.info(model)

    # prepare for (multi-device) GPU training
    n_gpu = 1
    device, device_ids = prepare_device(n_gpu)
    model = model.to(device)
    if len(device_ids) > 1:
        model = torch.nn.DataParallel(model, device_ids=device_ids)

    # get function handles of loss and metrics
    criterion = module_loss.crossentropyloss
    metrics = [module_metric.accuracy]

    optimizer = optim.Adam(model.parameters(), lr=.03, weight_decay=1e-4)
    #lr_scheduler = StepLR(optimizer, step_size=15, gamma=0.001)
    #optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, verbose=False, three_phase=True,  max_lr=.03, pct_start=5/24, epochs=24, steps_per_epoch=len(data_loader))

    trainer = Trainer(model, criterion, metrics, optimizer,
                      config=config,
                      device=device,
                      data_loader=data_loader,
                      valid_data_loader=valid_data_loader,
                      lr_scheduler=scheduler)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 100 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train Epoch: 1 last_lr_used  0.001200  [0/1000 (0%)] Loss: 7.089025
Train Epoch: 1 last_lr_used  0.001200  [10/1000 (1%)] Loss: 5.614312
Train Epoch: 1 last_lr_used  0.001201  [20/1000 (2%)] Loss: 5.307779
Train Epoch: 1 last_lr_used  0.001203  [30/1000 (3%)] Loss: 5.214440
Train Epoch: 1 last_lr_used  0.001205  [40/1000 (4%)] Loss: 5.102050
Train Epoch: 1 last_lr_used  0.001207  [50/1000 (5%)] Loss: 5.130153
Train Epoch: 1 last_lr_used  0.001211  [60/1000 (6%)] Loss: 5.116416
Train Epoch: 1 last_lr_used  0.001214  [70/1000 (7%)] Loss: 5.029659
Train Epoch: 1 last_lr_used  0.001219  [80/1000 (8%)] Loss: 5.142449
Train Epoch: 1 last_lr_used  0.001224  [90/1000 (9%)] Loss: 5.052190
Train Epoch: 1 last_lr_used  0.001229  [100/1000 (10%)] Loss: 4.962533
Train Epoch: 1 last_lr_used  0.001235  [110/1000 (11%)] Loss: 5.047200
Train Epoch: 1 last_lr_used  0.001242  [120/1000 (12%)] Loss: 5.031169
Train Epoch: 1 last_lr_used  0.001249  [130/1000 (13%)] Loss: 4.885757
Train Epoch: 1 last_lr_used

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

def denormalize(images, means, stds):
    means = torch.tensor(means).reshape(1, 3, 1, 1)
    stds = torch.tensor(stds).reshape(1, 3, 1, 1)
    return images * stds + means

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([]); ax.set_yticks([])
        denorm_images = denormalize(images, *stats)
        ax.imshow(make_grid(denorm_images[:64], nrow=8).permute(1, 2, 0).clamp(0,1))
        break

In [ ]:
show_batch(valid_data_loader)


In [49]:
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

Cloning into 'IMagenet'...
remote: Enumerating objects: 120594, done.
remote: Total 120594 (delta 0), reused 0 (delta 0), pack-reused 120594
Receiving objects: 100% (120594/120594), 212.68 MiB | 25.22 MiB/s, done.
Resolving deltas: 100% (1115/1115), done.
Checking out files: 100% (120206/120206), done.
test  train  val  wnids.txt  words.txt


In [53]:
import time
import scipy.ndimage as nd
import numpy as np
import cv2
im = cv2.imread('astronaut.png')

path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open( path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict
  
def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open( path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])      
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [cv2.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i))) for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open( path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(cv2.imread( path + 'val/images/{}'.format(img_name)))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)
  
train_data, train_labels, test_data, test_labels = get_data(get_id_dictionary())

print( "train data shape: ",  train_data.shape )
print( "train label shape: ", train_labels.shape )
print( "test data shape: ",   test_data.shape )
print( "test_labels.shape: ", test_labels.shape )

starting loading data
finished loading data, in 15.017764568328857 seconds
train data shape:  (100000, 64, 64, 3)
train label shape:  (100000, 200)
test data shape:  (10000, 64, 64, 3)
test_labels.shape:  (10000, 200)


In [78]:
np.concatenate((test_data , train_data[:20000]))
np.concatenate((test_labels , train_labels[:20000]))

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [81]:
len(train_labels)

100000